# Big Data Modeling and Management Assigment - Homework 1

# Submission

GROUP NUMBER: **XXXXXX** - please add your group number into the file name

GROUP MEMBERS:

|STUDENT NAME|STUDENT NUMBER|
|---|---|
|XXXXXX|XXXXXX|
|XXXXXX|XXXXXX|
|XXXXXX|XXXXXX|
|XXXXXX|XXXXXX|

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating related information. For this first project we will be taking a graph database to analyse beer and breweries!   

The project datasets are based on [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. 

### Problem description

Imagine you are working in the Data Management department of Analytics company.
Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions while adjusting the database to meet the needs of your colleagues.
Please record and keep track of your database changes, and submit the file with all cells run and with the output shown.

### Questions

1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]
2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
3. Analytics department requires the following information for the biweekly reporting: [5 points]
    1. How many reviews has the beer with the most reviews?
    2. Which three users wrote the most reviews about beers?
    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
    4. Which top three breweries produce the largest variety of beer styles?
    5. Which country produces the most beer styles?
4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
    2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.
5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?
 
### Groups  

Groups should have 4 people maximum. Please mark which group you are here: https://shorturl.at/zE0QP 

### Submission      

The code used to produce the results and to-the-point explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 18, 2025**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and efficiency of the query (based on database schema).  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/PATH/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/PATH/Neo4Jplugins`.
- Change the code below accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW2025 -p 7474:7474 -p 7687:7687 -d -v "c:\PATH\Neo4JPlugins":/plugins -v "c:\PATH\Neo4JHWData\data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

If the neo4j browser fails to load gds plugins, run the following in the Command Prompt before creating the container again:
`// Remove stopped containers //
docker container prune -f
// Remove unused images //
docker image prune -a -f
// Remove unused volumes //
docker volume prune -f
// Remove unused networks //
docker network prune -f
// Remove all unused resources in one command //
docker system prune -a -f`

In [7]:
#docker run --name Neo4JHW2025 -p 7474:7474 -p 7687:7687 -d -v "c:\PATH\Neo4JPlugins":/plugins -v "c:\PATH\Neo4JHWData\data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5

In [8]:
from neo4j import GraphDatabase
from pprint import pprint

In [9]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [10]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

In [11]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

## 1 Overview of the data


In [28]:
query = """
        CALL db.schema.visualization();
    """

result = execute_read(driver, query)

pprint(result)

[<Record nodes=[<Node element_id='-19' labels=frozenset({'REVIEWS'}) properties={'name': 'REVIEWS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-18' labels=frozenset({'BEERS'}) properties={'name': 'BEERS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-15' labels=frozenset({'COUNTRIES'}) properties={'name': 'COUNTRIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-17' labels=frozenset({'BREWERIES'}) properties={'name': 'BREWERIES', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-20' labels=frozenset({'STYLE'}) properties={'name': 'STYLE', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-16' labels=frozenset({'CITIES'}) properties={'name': 'CITIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-21' labels=frozenset({'USER'}) properties={'name': 'USER', 'indexes': ['name'], 'constraints': []}>] relationships=[<Relationship element_id='-17' nodes=(<Node element_id='-18' labels=frozenset({'BEERS'}) properties={

list all unique labels

In [29]:
query = """
MATCH (n) 
RETURN DISTINCT labels(n) AS NodeTypes;
"""

result = execute_read(driver, query)
pprint(result)  #


[<Record NodeTypes=['COUNTRIES']>,
 <Record NodeTypes=['CITIES']>,
 <Record NodeTypes=['BREWERIES']>,
 <Record NodeTypes=['BEERS']>,
 <Record NodeTypes=['REVIEWS']>,
 <Record NodeTypes=['STYLE']>,
 <Record NodeTypes=['USER']>]


list all relationsships

In [30]:
query = """
MATCH ()-[r]-() 
RETURN DISTINCT type(r) AS RelationshipTypes;
"""

result = execute_read(driver, query)
pprint(result)  # Print relationship types


[<Record RelationshipTypes='IN'>,
 <Record RelationshipTypes='BREWED'>,
 <Record RelationshipTypes='HAS_STYLE'>,
 <Record RelationshipTypes='REVIEWED'>,
 <Record RelationshipTypes='POSTED'>]


In [15]:
query = """
CALL db.stats.retrieve('GRAPH COUNTS');
"""

result = execute_read(driver, query)
pprint(result)  # Print database stats


[<Record section='GRAPH COUNTS' data={'relationships': [{'count': 5856205}, {'relationshipType': 'REVIEWED', 'count': 2537991}, {'relationshipType': 'REVIEWED', 'startLabel': 'BEERS', 'count': 2537991}, {'relationshipType': 'REVIEWED', 'count': 2537991, 'endLabel': 'REVIEWS'}, {'relationshipType': 'BREWED', 'count': 358873}, {'relationshipType': 'BREWED', 'startLabel': 'BREWERIES', 'count': 358873}, {'relationshipType': 'BREWED', 'count': 358873, 'endLabel': 'BEERS'}, {'relationshipType': 'IN', 'count': 62424}, {'relationshipType': 'IN', 'count': 12077, 'endLabel': 'COUNTRIES'}, {'relationshipType': 'IN', 'startLabel': 'CITIES', 'count': 12077}, {'relationshipType': 'IN', 'count': 50347, 'endLabel': 'CITIES'}, {'relationshipType': 'IN', 'startLabel': 'BREWERIES', 'count': 50347}, {'relationshipType': 'HAS_STYLE', 'count': 358873}, {'relationshipType': 'HAS_STYLE', 'startLabel': 'BEERS', 'count': 358873}, {'relationshipType': 'HAS_STYLE', 'count': 358873, 'endLabel': 'STYLE'}, {'relatio

In [16]:
query = """
CALL db.stats.retrieve('GRAPH COUNTS');
"""

result = execute_read(driver, query)
pprint(result)  # Print database stats


[<Record section='GRAPH COUNTS' data={'relationships': [{'count': 5856205}, {'relationshipType': 'REVIEWED', 'count': 2537991}, {'relationshipType': 'REVIEWED', 'startLabel': 'BEERS', 'count': 2537991}, {'relationshipType': 'REVIEWED', 'count': 2537991, 'endLabel': 'REVIEWS'}, {'relationshipType': 'BREWED', 'count': 358873}, {'relationshipType': 'BREWED', 'startLabel': 'BREWERIES', 'count': 358873}, {'relationshipType': 'BREWED', 'count': 358873, 'endLabel': 'BEERS'}, {'relationshipType': 'IN', 'count': 62424}, {'relationshipType': 'IN', 'count': 12077, 'endLabel': 'COUNTRIES'}, {'relationshipType': 'IN', 'startLabel': 'CITIES', 'count': 12077}, {'relationshipType': 'IN', 'count': 50347, 'endLabel': 'CITIES'}, {'relationshipType': 'IN', 'startLabel': 'BREWERIES', 'count': 50347}, {'relationshipType': 'HAS_STYLE', 'count': 358873}, {'relationshipType': 'HAS_STYLE', 'startLabel': 'BEERS', 'count': 358873}, {'relationshipType': 'HAS_STYLE', 'count': 358873, 'endLabel': 'STYLE'}, {'relatio

count each node

In [17]:
query = """
MATCH (n) 
RETURN labels(n)[0] AS NodeType, count(n) AS Count
ORDER BY Count DESC;
"""

result = execute_read(driver, query)
pprint(result)  # Print node type counts


[<Record NodeType='REVIEWS' Count=2549252>,
 <Record NodeType='BEERS' Count=417746>,
 <Record NodeType='USER' Count=123935>,
 <Record NodeType='BREWERIES' Count=100694>,
 <Record NodeType='CITIES' Count=23330>,
 <Record NodeType='COUNTRIES' Count=400>,
 <Record NodeType='STYLE' Count=113>]


count relationsships

In [18]:
query = """
MATCH ()-[r]->() 
RETURN type(r) AS RelationshipType, count(r) AS Count
ORDER BY Count DESC;
"""

result = execute_read(driver, query)
pprint(result)  # Print relationship counts


[<Record RelationshipType='POSTED' Count=2538044>,
 <Record RelationshipType='REVIEWED' Count=2537991>,
 <Record RelationshipType='BREWED' Count=358873>,
 <Record RelationshipType='HAS_STYLE' Count=358873>,
 <Record RelationshipType='IN' Count=62424>]


### DATA CLEANING


city in city dosent make sense

In [19]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        session.execute_write(lambda tx, query: tx.run(query), query)

query = """
MATCH (c:CITIES)-[r:IN]->(c:CITIES)
DELETE r;
"""

execute_write(driver, query)
print("Self-referencing city relationships deleted successfully.")


Self-referencing city relationships deleted successfully.


In [20]:
query = """
MATCH (c:CITIES)-[r:IN]->(c:CITIES)
RETURN c, r
"""

result = execute_read(driver, query)
pprint(result)  # Should return an empty list if deletion was successful


[]


## 3. Queries

beer with the most reviews

In [35]:
query = """
MATCH (b:BEERS)<-[:REVIEWED]-(r:REVIEWS)
RETURN b.name AS Beer, COUNT(r) AS ReviewCount
ORDER BY ReviewCount DESC
LIMIT 1;
"""

result = execute_read(driver, query)
pprint(result)  # Should return the beer with the most reviews


[]


return the top 3 users with most reviews

In [36]:
query = """
MATCH (u:USER)-[:POSTED]->(r:REVIEWS)
RETURN u.name AS User, COUNT(r) AS ReviewCount
ORDER BY ReviewCount DESC
LIMIT 3;
"""

result = execute_read(driver, query)
pprint(result)  # Should return the top 3 users with most reviews


[]


beers with specific descriptions

In [37]:
query = """
MATCH (b:BEERS)<-[:REVIEWED]-(r:REVIEWS)
WHERE r.text CONTAINS 'fruit' OR r.text CONTAINS 'complex' 
   OR r.text CONTAINS 'nutty' OR r.text CONTAINS 'dark'
RETURN DISTINCT b.name AS Beer;
"""

result = execute_read(driver, query)
pprint(result)  # Should return beers with those descriptions


[]


3 breweries with the most beer styles

In [33]:
query = """
MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
RETURN br.name AS Brewery, COUNT(DISTINCT s.name) AS StyleCount
ORDER BY StyleCount DESC
LIMIT 3;
"""

result = execute_read(driver, query)
pprint(result)  # Should return top 3 breweries with the most beer styles


[<Record Brewery='Iron Hill Brewery & Restaurant' StyleCount=94>,
 <Record Brewery='Rock Bottom Restaurant & Brewery' StyleCount=93>,
 <Record Brewery='Goose Island Beer Co.' StyleCount=88>]


country producing the most beer styles

In [25]:
query = """
MATCH (co:COUNTRIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
RETURN co.name AS Country, COUNT(DISTINCT s.name) AS StyleCount
ORDER BY StyleCount DESC
LIMIT 1;
"""

result = execute_read(driver, query)
pprint(result)  # Should return the country producing the most beer styles


[]


country producing the most beer styles

In [38]:
query = """
MATCH (r:REVIEWS), (b:BEERS)
WHERE r.beer_id = b.id
MERGE (r)-[:REVIEWED]->(b);
"""
result = execute_read(driver, query)
pprint(result) # Should return the country producing the most beer styles


ClientError: {code: Neo.ClientError.Statement.AccessMode} {message: Writing in read access mode not allowed. Attempted write to internal graph 0 (neo4j)}